# 0. Setup
Load bbrest and environment variables.

In [1]:
from bbrest import BbRest
from os import getenv as ge
from dotenv import load_dotenv
load_dotenv()

url = ge('url')
key = ge('key')
secret = ge('secret')

bb = BbRest(url=url,
            key=key,
            secret=secret)

In [1]:
import uuid

In [7]:
str(uuid.uuid1())

'c6975536-cdac-11ea-857a-6c96cfd83ab1'

# 1. Getting the code manually
* Use existing connection to generate an auth URL.
* Click on URL and log in 
* Grab the 'code' string from the url
* Paste the code and the redirect_uri into a new BbRest object

### Use existing connection to generate an auth url

In [2]:
redirect_uri = 'https://test.blackboard.com'
bb.get_auth_url(scope='read', redirect_uri=redirect_uri)

'https://kubbrest.zapto.org/webapps/login/?new_loc=%2Fwebapps%2Fapi-gateway%2Foauth2%2Fauthorizationcode%3Fresponse_type%3Dcode%26client_id%3Dd279753a-8319-4910-8b27-0ed5de84bcaf%26redirect_uri%3Dhttps%253A%252F%252Ftest.blackboard.com%26scope%3Dread%26state%3DDC1067EE-63B9-40FE-A0AD-B9AC069BF4B0'

### Click on URL and log in
### Grab the 'code' string from the url

![SegmentLocal](images/LoginExample.gif "new login")

### Paste the code and the redirect_uri into a new BbRest object

In [3]:
code="Mtj9TDbWDsf92aTrOM72pZ2kr7COK5FV"

In [4]:
bb2 = BbRest(url=url,
             key=key,
             secret=secret,
             code=code,
             redirect_uri=redirect_uri)

# 2. New Features
All BbRest objects now have a token_info dictionary. This contains all information recieved back from a token. 

This is stored so that we can do refreshes, and ensure BbRest is authenticated using the user and scopes we expect.

If elevated privelleges have been granted by a user to a REST application - it is difficult to reduce these, so typically start with Read scope.

Token info also does not update until a token has to be refreshed, so - the expiration() function is still more useful at letting you know how much time individual BbRest objects are authenticated for.

In [5]:
bb.token_info

{'access_token': 'r9u1az3lA38fuuzkIq4P8ccf88YJK9CT',
 'token_type': 'bearer',
 'expires_in': 2756}

In [6]:
bb2.token_info

{'access_token': 'ADFN04MfXXtEW28Q67dSyROS3XzJgxhP',
 'token_type': 'bearer',
 'expires_in': 2412,
 'scope': 'read',
 'user_id': '1b8488442b7b4a979bd33ec8dec0d939'}

In [7]:
bb2.expiration()

'in 40 minutes'

In [8]:
# User permissions
bb2.GetUser('uuid:1b8488442b7b4a979bd33ec8dec0d939').json()

{'id': '_10_1',
 'userName': 'testing2',
 'educationLevel': 'Unknown',
 'gender': 'Unknown',
 'modified': '2020-07-06T19:15:34.056Z',
 'institutionRoleIds': ['STUDENT'],
 'systemRoleIds': ['User'],
 'name': {'given': 'Test', 'family': 'Testerton'}}

In [9]:
# Admin permissions
bb.GetUser('uuid:1b8488442b7b4a979bd33ec8dec0d939').json()

{'id': '_10_1',
 'uuid': '1b8488442b7b4a979bd33ec8dec0d939',
 'externalId': 'testing2',
 'dataSourceId': '_2_1',
 'userName': 'testing2',
 'educationLevel': 'Unknown',
 'gender': 'Unknown',
 'created': '2020-07-06T18:55:55.992Z',
 'modified': '2020-07-06T19:15:34.056Z',
 'lastLogin': '2020-07-06T19:15:34.055Z',
 'institutionRoleIds': ['STUDENT'],
 'systemRoleIds': ['User'],
 'availability': {'available': 'Yes'},
 'name': {'given': 'Test', 'family': 'Testerton'}}

In [10]:
# User permissions 
#In the example below - this is an Admin user
bb2.GetUser('m500d520').json()

{'status': 404, 'message': 'User does not exist:Requested user is not found.'}

In [11]:
# Admin permissions
bb.GetUser('m500d520').json()

{'id': '_6_1',
 'uuid': '1afc494e687940e9a07944785c9ebd5b',
 'externalId': 'm500d520',
 'dataSourceId': '_2_1',
 'userName': 'm500d520',
 'educationLevel': 'Unknown',
 'gender': 'Male',
 'created': '2020-06-24T19:51:41.208Z',
 'modified': '2020-07-06T16:16:33.579Z',
 'lastLogin': '2020-07-06T16:16:33.578Z',
 'institutionRoleIds': ['STAFF'],
 'systemRoleIds': ['SystemAdmin'],
 'availability': {'available': 'Yes'},
 'name': {'given': 'Matt', 'family': 'Deakyne'},
 'contact': {'email': 'm.d@ku.edu'}}

In [12]:
# Read permissions
bb2.UpdateUser('uuid:1b8488442b7b4a979bd33ec8dec0d939', payload={'password':'test'}).json()

{'status': 403,
 'message': 'Application not authorized to perform PATCH requests.'}

# 3. Refreshing a token 

### Non-3lo refresh example

In [13]:
bb.token_info

{'access_token': 'r9u1az3lA38fuuzkIq4P8ccf88YJK9CT',
 'token_type': 'bearer',
 'expires_in': 2756}

In [14]:
bb.refresh_token()

In [15]:
bb.token_info

{'access_token': 'r9u1az3lA38fuuzkIq4P8ccf88YJK9CT',
 'token_type': 'bearer',
 'expires_in': 2713}

### Non-offline 3lo example
Do not refresh the token unless it's expired, as you will have to log in again.  

In [17]:
bb2.token_info

{'access_token': 'ADFN04MfXXtEW28Q67dSyROS3XzJgxhP',
 'token_type': 'bearer',
 'expires_in': 2412,
 'scope': 'read',
 'user_id': '1b8488442b7b4a979bd33ec8dec0d939'}

In [18]:
bb2.refresh_token()

Need to log in again at https://kubbrest.zapto.org/webapps/login/?new_loc=%2Fwebapps%2Fapi-gateway%2Foauth2%2Fauthorizationcode%3Fresponse_type%3Dcode%26client_id%3Dd279753a-8319-4910-8b27-0ed5de84bcaf%26redirect_uri%3Dhttps%253A%252F%252Ftest.blackboard.com%26scope%3Dread%26state%3DDC1067EE-63B9-40FE-A0AD-B9AC069BF4B0


In [19]:
code = "kJd8OCY2b3YUjpEFpahrgXgyxk95n8aw"
bb2 = BbRest(url=url,
             key=key,
             secret=secret,
             code=code,
             redirect_uri=redirect_uri)

In [20]:
bb2.token_info

{'access_token': 'ADFN04MfXXtEW28Q67dSyROS3XzJgxhP',
 'token_type': 'bearer',
 'expires_in': 2267,
 'scope': 'read',
 'user_id': '1b8488442b7b4a979bd33ec8dec0d939'}

### Offline 3lo example
If expired, BbRest will auto renew the token using the refresh_token workflow.  

In [21]:
redirect_uri = 'https://test.blackboard.com'
bb.get_auth_url(scope='read write offline', redirect_uri=redirect_uri)

'https://kubbrest.zapto.org/webapps/login/?new_loc=%2Fwebapps%2Fapi-gateway%2Foauth2%2Fauthorizationcode%3Fresponse_type%3Dcode%26client_id%3Dd279753a-8319-4910-8b27-0ed5de84bcaf%26redirect_uri%3Dhttps%253A%252F%252Ftest.blackboard.com%26scope%3Dread%2Bwrite%2Boffline%26state%3DDC1067EE-63B9-40FE-A0AD-B9AC069BF4B0'

In [22]:
code="RPTXLq80znEdMQDN5TzK506MjqfVVtxj"
bb3 = BbRest(url=url,
             key=key,
             secret=secret,
             code=code,
             redirect_uri=redirect_uri)

In [23]:
bb3.token_info

{'access_token': 'qk4UB1ynbsapWZt1FNkEJx3WpxVLLmto',
 'token_type': 'bearer',
 'expires_in': 2921,
 'refresh_token': '6f032f179ee748c3955ff20308462164:lVYLO8jtT3vIB0btfA9xX7qEeLBiCJnY',
 'scope': 'read write offline',
 'user_id': '6f032f179ee748c3955ff20308462164'}

In [24]:
bb3.refresh_token()

In [25]:
bb3.token_info

{'access_token': 'qk4UB1ynbsapWZt1FNkEJx3WpxVLLmto',
 'token_type': 'bearer',
 'expires_in': 2916,
 'refresh_token': '6f032f179ee748c3955ff20308462164:YjnEwdnQcMbdNyrSwW0cPBPwVW9cnmtz',
 'scope': 'read write offline',
 'user_id': '6f032f179ee748c3955ff20308462164'}

# 4. Automating the login process
* Create a user
* Write code using helium to login as that user and pull the code
* Automate everything

In [26]:
bb.CreateUser(payload={'userName':'testing4',
                       'name':{'given':'Test 4','family':'Testerton'},
                       'password':'xxxxxxx'}).json()

{'id': '_12_1',
 'uuid': 'c47436e553b840c0a9ddb9b450e1b1db',
 'externalId': 'testing4',
 'dataSourceId': '_2_1',
 'userName': 'testing4',
 'educationLevel': 'Unknown',
 'gender': 'Unknown',
 'created': '2020-07-06T19:27:10.478Z',
 'modified': '2020-07-06T19:27:10.478Z',
 'institutionRoleIds': ['STUDENT'],
 'systemRoleIds': ['User'],
 'availability': {'available': 'Yes'},
 'name': {'given': 'Test 4', 'family': 'Testerton'}}

In [ ]:
def 